# Analisi dati

### Analisi dei personaggi

Confronto fra i personaggi della web api e dello scraping, confrontare il nome in modo da porter collegare i dati dei personaggi ottenuti dallo scriping ai dati della web api


In [21]:
import pandas as pd
import re
from tqdm import tqdm
import json


In [19]:
api_characters_dataset = pd.read_csv("./Datasets/api_character_list.csv")
scraping_characters_dataset = pd.read_csv("./Datasets/scraping_personaggi_completo_filtered_relations.csv")

In [6]:
def matching_characters(first_name, second_name):
  first_name_without_paranthesys = re.sub("[\(\[].*?[\)\]]", "", first_name.strip()).strip()
  first_name_without_paranthesys = first_name_without_paranthesys.replace("'", " ").replace('"', " ").replace("-"," ").strip()
  second_name_without_paranthesys = re.sub(
      "[\(\[].*?[\)\]]", "", second_name.strip()).strip()
  secondt_name_without_paranthesys = second_name_without_paranthesys.replace(
      "'", " ").replace('"', " ").replace("-", " ").strip()
  return first_name_without_paranthesys == second_name_without_paranthesys


In [15]:
for idx, row in tqdm(scraping_characters_dataset.iterrows(), total=scraping_characters_dataset.shape[0]):
  proc_name = re.sub(
      "[\(\[].*?[\)\]]", "", row["Processed_Name"].strip()).strip()
  scraping_characters_dataset.at[idx, "Processed_Name"] = proc_name
scraping_characters_dataset.head(10)
scraping_characters_dataset.to_csv(
    "./Datasets/scraping_personaggi_completo_filtered_relations.csv")


100%|██████████| 3746/3746 [00:00<00:00, 22160.78it/s]


In [20]:
num_link = 0
for idx, row in tqdm(api_characters_dataset.iterrows(), total=api_characters_dataset.shape[0]):
  formatted_name = re.sub(
      "[\(\[].*?[\)\]]", "", row["name"].strip()).strip()
  querystring = "Processed_Name == '%s'" % (
      formatted_name.replace("'", " ").replace('"', " ").replace(" \u2020", ""))
  if len(scraping_characters_dataset.query(querystring)) != 0:
    num_link += 1
print("Numero di corrispondenze: %s"%(num_link))


100%|██████████| 1559/1559 [00:01<00:00, 1157.91it/s]

Numero di corrispondenze: 323


323 personaggi ottenuti dalla web api sono collegabili con i personaggi ottenuti tramite scraping

# Filtering relazioni

Identificazione preliminare delle relazioni che non trovano corrispondenza nella lista di personaggi

In [24]:
character_dataset_scraping = pd.read_csv("./Datasets/scraping_personaggi_completo_filtered_name.csv")
linked = 0
totale = 0
for idx, row in tqdm(character_dataset_scraping.iterrows(), total=character_dataset_scraping.shape[0]):
  try:
    relations = json.loads(row["Relazioni"])
    filtered_relations = []
    for relation in relations:
        totale += 1
        querystring = "Processed_Name == '%s'" % (
            relation[0].replace("'", " ").replace('"', " ").replace(" \u2020", ""))
        if len(character_dataset_scraping.query(querystring)) != 0:
          linked +=1 
  except Exception as e:
   None
print("Tenute %s relazioni su %s"%(linked, totale))


100%|██████████| 3746/3746 [00:46<00:00, 81.14it/s] 

Tenute 45309 relazioni su 61114
